In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-internship-20231/train.csv
/kaggle/input/nlp-internship-20231/test.csv


In [2]:
import random
import numpy as np
import torch

In [3]:
def seed_all(seed_value):
    random.seed(seed_value) 
    np.random.seed(seed_value) 
    torch.manual_seed(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True 
        torch.backends.cudnn.benchmark = False
        
seed = 42
seed_all(seed)

In [71]:
train_df = pd.read_csv('/kaggle/input/nlp-internship-20231/train.csv')
# train_df.head(5)
data_for_prediction_df = pd.read_csv('/kaggle/input/nlp-internship-20231/test.csv')

In [ ]:
train_df['word_len'].hist()

In [72]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_df['lib'])
train_df['label'] = le.transform(train_df['lib'])
# >>> le.inverse_transform([0, 0, 1, 2])

In [41]:
le.classes_

array(['collections', 'csv', 'datetime', 'django', 'flask', 'functools',
       'itertools', 'json', 'math', 'matplotlib', 'numpy', 'os', 'pandas',
       'random', 're', 'requests', 'scipy', 'selenium', 'sklearn',
       'subprocess', 'sys', 'tensorflow', 'time', 'urllib'], dtype=object)

In [73]:
train_df['title'] = train_df['title'].apply(lambda x: x.lower())
train_df.head()

,title,lib,id,label
0,is there a way to sort strings in alphabetical...,functools,84026,5
1,maintaining history by soft deleting the row o...,django,8930,3
2,wave on a string analysis,numpy,133474,10
3,regular expression find word but not if it's p...,re,34429,14
4,textblob - loop over articles to calculate pol...,pandas,82106,12


In [74]:
# for c in le.classes_:
libs = pd.DataFrame(np.vstack(train_df['title'].apply(lambda x: \
        [1 if c in x.split(' ') else 0 for c in le.classes_]).values), columns=le.classes_)
train_df = pd.concat((train_df, libs), axis=1)

In [76]:
train_df[train_df['datetime'] == 1]

,title,lib,id,label,collections,csv,datetime,django,flask,functools,...,re,requests,scipy,selenium,sklearn,subprocess,sys,tensorflow,time,urllib
67,how to add arrow annotations with an offset to...,pandas,46764,12,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70,convert timestamp string date to datetime python,pandas,43422,12,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
220,generate offsets for each row in a dataframe b...,pandas,78905,12,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
616,how to append gmt+ to datetime object in python,datetime,5072,2,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
858,sub-class python datetime object,datetime,12883,2,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133288,flagging overlapping datetime ranges,datetime,116777,2,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
133499,merge datetime series to multi-index dataframe,numpy,26661,10,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
133513,comparing a datetime dataframe with a period d...,pandas,78703,12,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
133834,extract time from datetime object and compare ...,datetime,32429,2,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# train_df = train_df.sample(10000)
train_df.head()

### NN

In [9]:
from transformers import BertModel, BertTokenizer, AdamW,  get_linear_schedule_with_warmup, set_seed
from transformers import DistilBertTokenizer, DistilBertModel
import torch.nn.functional as F
from torch.utils.data import DataLoader , Dataset
import torch.nn as nn 
import torch
import numpy as np
from sklearn.metrics import f1_score, precision_score
from sklearn.model_selection import StratifiedKFold
import random

import glob
import os
import pandas as pd

In [24]:
class ModelConfig:
    NB_EPOCHS = 5
    LR = 3e-5
    EPS = 1e-8
    MAX_LEN = 32
    N_SPLITS = 4
    TRAIN_BS = 64
    VALID_BS = 32
#     MODEL_NAME = 'cointegrated/rubert-tiny-sentiment-balanced'
#     TOKENIZER = BertTokenizer.from_pretrained('cointegrated/rubert-tiny-sentiment-balanced')
    MODEL_NAME = 'nbroad/ESG-BERT'
    TOKENIZER = BertTokenizer.from_pretrained('nbroad/ESG-BERT')
#     MODEL_NAME = 'carrassi-ni/bert-base-trec-question-classification'
#     TOKENIZER = BertTokenizer.from_pretrained('carrassi-ni/bert-base-trec-question-classification')


In [102]:
n_labels = len(train_df['label'].value_counts())
model = LibClassifier(n_labels)
model.cuda()
pass
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# train(model, train_data, test_data, ModelConfig.NB_EPOCHS, device, 'final_model')
# pickle.dump(model, open('final_model.sav', 'wb'))

Some weights of the model checkpoint at nbroad/ESG-BERT were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [109]:
class LibDataset(Dataset):
    def __init__(self, texts, targets=None, le_classes=None, n_classes=24, is_test=False):
        self.texts = texts
        self.targets = targets
        self.le_classes = le_classes
        self.n_classes = n_classes
        self.is_test = is_test
        self.tokenizer = ModelConfig.TOKENIZER
        self.max_len = ModelConfig.MAX_LEN
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        text = ' '.join(text.split())
       
        inputs = self.tokenizer(
                            text,
                            add_special_tokens=True,
                            max_length=self.max_len,
                            padding="max_length" ,
                            truncation = True ,
                            pad_to_max_length=True, 
                            )
        
        ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
        mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)
#         token_type = torch.tensor(inputs['token_type_ids'], dtype=torch.long)
     
        le_classes = torch.tensor(self.le_classes[idx], dtype=torch.float32)
        if self.is_test:
            
            return {
                'ids': ids,
                'mask': mask,
                'le_classes': le_classes
#                 'token_type': token_type,
            }
        else:    
#             one_hot = F.one_hot(torch.tensor(self.targets[idx], dtype=torch.long), self.n_classes)
            targets = torch.tensor(self.targets[idx], dtype=torch.long)
            return {
                'ids': ids,
                'mask': mask,
                'le_classes': le_classes,
#                 'token_type': token_type,
                'targets': targets
            }


In [94]:
# train_df[le.classes_]

In [134]:
def LibDataloader(df, le_classes, batch_size, is_test=False, n_classes=24):
    dataset = LibDataset(texts=df["title"].values,
                         targets=df["label"].values,
                         le_classes=df[le_classes].values,
                         is_test=False,
                         n_classes = n_classes)
    dataloader = DataLoader(dataset, batch_size, shuffle=False)
    return dataloader

In [121]:
class LibClassifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained(ModelConfig.MODEL_NAME, return_dict=False)
        self.drop = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(24, 768)
        self.drop1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(768, 768)
        self.out = nn.Linear(1536, n_classes)
        
        
    def forward(self, input_ids, attention_mask, le_classes):
        
        _, output = self.bert(input_ids, attention_mask)
#         print(output)
        output = self.drop(output)
        
        output_2 = self.fc1(le_classes)
        output_2 = self.drop1(output_2)
        output_2 = self.fc2(output_2)
        
        output = torch.cat((output, output_2), 1)
        output = self.out(output)
    
        return output

In [14]:
n_labels = len(train_df['label'].value_counts())
print(n_labels)
model = LibClassifier(n_labels)

24


Some weights of the model checkpoint at nbroad/ESG-BERT were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
def loss_fn(outputs, labels):
    return nn.CrossEntropyLoss()(outputs, labels)

def precision_micro(outputs, targets):
    pred = np.argmax(outputs.cpu().detach().numpy(), axis=1).flatten()
    targets = targets.cpu().detach().numpy()
    return precision_score(pred, targets, average='micro')

In [16]:
def yield_optimizer(model):
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]

    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    return torch.optim.AdamW(optimizer_parameters, lr=ModelConfig.LR, eps=ModelConfig.EPS) 

In [118]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model.train()
    losses = []
    correct_predictions = []

    for step , d in enumerate(data_loader):
        input_ids = d['ids'].to(device) 
#         token_type_ids = d['token_type'].to(device)
        attention_mask = d['mask'].to(device)
        targets = d['targets'].to(device)
        le_classes = d['le_classes'].to(device)
        outputs = model(
            input_ids ,
            attention_mask ,
            le_classes
#             token_type_ids,
            )
        
        loss = loss_fn(outputs, targets)
        correct_predictions.append(precision_micro(outputs, targets))
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
    return np.mean(correct_predictions, axis=0), np.mean(losses)

In [123]:
def eval_model (model, data_loader, loss_fn, device, n_examples):
    model.eval()
  
    losses = []
    correct_predictions = []

    with torch.no_grad():
        for step , d in enumerate(data_loader):
            
            input_ids = d['ids'].to(device)
#             token_type_ids = d['token_type'].to(device)
            attention_mask = d['mask'].to(device)
            le_classes = d['le_classes'].to(device)
            targets = d['targets'].to(device)
        
            outputs = model(
                   input_ids ,
                attention_mask ,
                le_classes
#                 token_type_ids,
                )

            loss = loss_fn(outputs , targets)
            correct_predictions.append(precision_micro(outputs, targets))
            losses.append(loss.item())

    return np.mean(correct_predictions, axis=0), np.mean(losses)

In [ ]:
train_df.head(5)

In [25]:
def train(model, df, test_df, epochs, device, model_name):
    best_accuracy = 0
    
    test_data_loader = LibDataloader(test_df, ModelConfig.VALID_BS)
    
    for epoch in range(epochs):
        # if epoch % 10 == 0:
        print(f'Epoch {epoch + 1}')

        kf = StratifiedKFold(n_splits=ModelConfig.N_SPLITS, random_state=seed, shuffle=True)

        for step, (train, valid ) in enumerate(kf.split(df , df["label"])) :

            train_data_loader = LibDataloader(df.iloc[train], ModelConfig.TRAIN_BS)
            validation_data_loader = LibDataloader(df.iloc[valid], ModelConfig.VALID_BS)

            nb_train_steps = int(len(train_data_loader) / ModelConfig.TRAIN_BS * epochs)
            optimizer = yield_optimizer(model)
            scheduler = get_linear_schedule_with_warmup(
                                        optimizer,
                                        num_warmup_steps=0,
                                        num_training_steps=nb_train_steps)

            train_acc, train_loss = train_epoch(model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df.iloc[train])) 
            val_acc, val_loss = eval_model(model, validation_data_loader, loss_fn, device, len(df.iloc[valid]))

            test_acc, _ = eval_model(model, test_data_loader, loss_fn, device, len(test_df))
            
            if  test_acc > best_accuracy:
                torch.save(model.state_dict(), model_name + '.bin')
                best_accuracy = test_acc
                print(f"Best accuracy {best_accuracy}")
            
            
            


In [115]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(train_df, train_size=0.9, stratify=train_df['label'])
# train_data, val_data = train_test_split(train_data, train_size=0.9, stratify=train_data['label'])

In [127]:
n_labels = len(train_df['label'].value_counts())
model = LibClassifier(n_labels)
model.cuda()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train(model, train_data, test_data, 4, device, 'final_model')

Some weights of the model checkpoint at nbroad/ESG-BERT were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1
Best accuracy 0.34363674533722266
Best accuracy 0.4476929196499602
Best accuracy 0.4947212277910369
Best accuracy 0.5161677936886767
Epoch 2
Best accuracy 0.5259380800848581
Best accuracy 0.5366641253425263
Best accuracy 0.540942941748431


KeyboardInterrupt: 

In [128]:
torch.save(model.state_dict(), 'final_model.bin')

In [129]:
from IPython.display import FileLink
FileLink(r'final_model.bin')

/kaggle/working/final_model.bin

In [131]:
from tqdm import tqdm

def get_predictions(model, df, le_classes):
    proba = []    
    model = model.eval()

    predictions = []

    test_data_loader = LibDataloader(df, le_classes, 1)
    with torch.no_grad():
        for d in tqdm(test_data_loader):
            
            input_ids = d["ids"].to(device)
            attention_mask = d["mask"].to(device)
            le_classes = d['le_classes'].to(device)
            outputs = model(
                            input_ids,
                            attention_mask ,
                            le_classes
                            )
            proba.append(torch.argmax(outputs).flatten().cpu().numpy())
    return np.array(proba).flatten()

In [130]:
def preprocess_data(df, le):
    df['title'] = df['title'].apply(lambda x: x.lower())
    df = df.dropna()
    libs = pd.DataFrame(np.vstack(df['title'].apply(lambda x: \
        [1 if c in x.split(' ') else 0 for c in le.classes_]).values), columns=le.classes_)
    df = pd.concat((df, libs), axis=1)
    df['sym_len'] = df['title'].apply(len)
    df['word_len'] = df['title'].apply(lambda x: len(x.split()))
    return df

for_prediction_df = pd.read_csv('/kaggle/input/nlp-internship-20231/test.csv')
for_prediction_df = preprocess_data(for_prediction_df, le)
for_prediction_df['label'] = -1

In [135]:
sample_submission = for_prediction_df[['id']]
sample_submission['lib'] = le.inverse_transform(get_predictions(model, for_prediction_df, le.classes_))
sample_submission.to_csv('./submission.csv')

100%|██████████| 33506/33506 [04:32<00:00, 122.79it/s]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [139]:
from IPython.display import FileLink
FileLink(r'./submission.csv')

/kaggle/working/submission.csv

In [83]:
# def TestDataloader(df, batch_size, is_test=True):
#     dataset = LibDataset(df, is_test)
#     dataloader = DataLoader(dataset, batch_size, shuffle=False)
#     return dataloader

def get_raw_predictions(model, df):
    proba = []    
    model = model.eval()


    test_data_loader = LibDataloader(df, 1)
    with torch.no_grad():
        for d in test_data_loader:
            
            input_ids = d["ids"].to(device)
            attention_mask = d["mask"].to(device)
            
            outputs = model(
                            input_ids,
                            attention_mask ,
                            )
            proba.append(outputs[0].cpu().numpy())
    return np.array(proba)

def get_predictions(model, df):
    proba = []    
    model = model.eval()

    predictions = []

#     data_loader = TestDataloader(df, 1)
    test_data_loader = LibDataloader(df, 1)
    with torch.no_grad():
        for d in test_data_loader:
            
            input_ids = d["ids"].to(device)
            attention_mask = d["mask"].to(device)
#             token_type_ids = d["token_type"].to(device)
            
            outputs = model(
                            input_ids,
                            attention_mask ,
#                             token_type_ids
                            )
            proba.append(torch.argmax(outputs).flatten().cpu().numpy())
    return np.array(proba).flatten()

# preds = get_predictions(model, test_data)

# print(classification_report(test_data['label'], preds))

In [ ]:
cm = confusion_matrix(test_data['label'], preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(10)
fig.set_figheight(10) 

In [ ]:
import pickle
pickle.dump(model, open('final_model.sav', 'wb'))